In [3]:
import pandas as pd
import numpy as np
import warnings
from functools import partial

In [4]:
from ViewsPandas.extensions import *


In [43]:
cx = fetch_ids_df('country_month')

In [44]:
cx.index.name

In [45]:
cx = cx.set_index('id')

In [51]:
'","'.join(['xxx','ppp','aaa','ff'])

'xxx","ppp","aaa","ff'

In [29]:
cx['muie'] = 1
cx['pula'] = 'real'

In [30]:
cx

,month_id,country_id,muie,pula
id,,,,
6173,365,5,1,real
7625,365,6,1,real
9077,365,7,1,real
13433,365,10,1,real
16337,365,12,1,real
...,...,...,...,...
238551,207,167,1,real
244490,218,183,1,real
270921,421,237,1,real


In [34]:
other_ids = [i for i in list(cx.columns) if '_id' in i.lower()]

In [41]:
cx.drop(other_ids,axis=1)

,muie,pula
id,,
6173,1,real
7625,1,real
9077,1,real
13433,1,real
16337,1,real
...,...,...
238551,1,real
244490,1,real
270921,1,real


In [4]:
xx = CMAccessor.new_structure()
xx

,c_id,month_id
0,5,365
1,6,365
2,7,365
3,10,365
4,12,365
...,...,...
278641,167,207
278642,183,218
278643,237,421
278644,246,406


In [12]:
pd.DataFrame(fetch_ids_df('priogrid_month').rename(columns={'id': 'pg_id'})[['pg_id','month_id']])

,pg_id,month_id
0,105478131,495
1,105478132,496
2,105478133,497
3,105478134,498
4,105478135,499
...,...,...
7223395,189066848,252
7223396,189066849,252
7223397,189066850,252
7223398,189066851,252


In [16]:
africa = CAccessor.new_structure()
return africa[africa.c.in_africa]
africa

,c_id
38,40
39,41
40,42
41,43
44,47
45,48
46,49
47,50
48,52
49,53


In [6]:
xx['okish'] = pd.DataFrame.c.soft_validate(xx)

In [7]:
xx.cm.db_id_id()

AttributeError: 'CMAccessor' object has no attribute 'db_id_id'

In [10]:
xx.month_id.max()

1452

In [3]:
from ViewsPandas.scratch import fetch_ids



In [4]:
pd.DataFrame.c.new_structure()

,c_id
0,1
1,2
2,3
3,4
4,5
...,...
215,251
216,252
217,253
218,254


In [6]:
pd.DataFrame.cm.new_africa()

,c_id,month_id
150,50,50
178,57,57
187,53,53
268,54,54
303,52,52
419,81,81
1813,40,40
1814,48,48
2010,49,49
2037,55,55


In [4]:
av_c, extra = fetch_ids('country')

In [21]:
set(av_c) == set(range(1,256))

False

In [6]:
a = set([1,2,3])
b = set([1,2,3,4,5])

In [7]:
Country(7).in_me

False

In [7]:
test_c = pd.DataFrame({'iso':['ITA','SWE','NOR','ROU','ISR'],'value':[1.1,2.1,2.2,1.7,3]})

In [8]:
test_c

,iso,value
0,ITA,1.1
1,SWE,2.1
2,NOR,2.2
3,ROU,1.7
4,ISR,3.0


In [9]:
new_df = pd.DataFrame.c.from_iso(test_c)

In [10]:
new_df

,iso,value,c_id
0,ITA,1.1,85
1,SWE,2.1,66
2,NOR,2.2,64
3,ROU,1.7,116
4,ISR,3.0,218


In [11]:
new_df.cm.expand_country_months()

,c_id,month_id,iso,value
0,85,1,ITA,1.1
1,85,2,ITA,1.1
2,85,3,ITA,1.1
3,85,4,ITA,1.1
4,85,5,ITA,1.1
...,...,...,...,...
3055,218,608,ISR,3.0
3056,218,609,ISR,3.0
3057,218,610,ISR,3.0
3058,218,611,ISR,3.0


In [12]:
new_df[new_df.iso.isin(['ITA','ROU'])]

,iso,value,c_id
0,ITA,1.1,85
3,ROU,1.7,116


In [13]:
test_cm = pd.DataFrame({'iso':['ITA','SWE','NOR','ROU'],'month_id':[401,401,404,404],'value':[1.1,2.1,2.2,1.7]})
test_cm

,iso,month_id,value
0,ITA,401,1.1
1,SWE,401,2.1
2,NOR,404,2.2
3,ROU,404,1.7


In [19]:
from ViewsPandas.extensions import *
from ViewsPandas.DBWriter import DBWriter


test_cm = pd.DataFrame({'iso':['ITA','SWE','NOR','ROU'],'month_id':[401,401,404,404],'value':[1.1,2.1,2.2,1.7]})

cm_writer = DBWriter(test_cm, 'cm')
df = pd.DataFrame(cm_writer.publish())

In [20]:
df

,table,column_name,sa_column,type,pkey,min_value,max_value,mean_value
0,reign_cm,month_id,reign_cm.month_id,<class 'int'>,False,1.0,612.0,319.334587
1,reign_cm,country_id,reign_cm.country_id,<class 'int'>,False,1.0,246.0,105.342297
2,reign_cm,elected,reign_cm.elected,<class 'float'>,False,0.0,1.0,0.713617
3,reign_cm,age,reign_cm.age,<class 'float'>,False,18.0,95.0,58.840612
4,reign_cm,male,reign_cm.male,<class 'float'>,False,0.0,1.0,0.954837
...,...,...,...,...,...,...,...,...
108,country_month,acled_fat_pr_c3,country_month.acled_fat_pr_c3,<class 'int'>,False,NaN,NaN,NaN
109,country_month,acled_count_pr_c4,country_month.acled_count_pr_c4,<class 'int'>,False,NaN,NaN,NaN
110,country_month,acled_fat_pr_c4,country_month.acled_fat_pr_c4,<class 'int'>,False,NaN,NaN,NaN
111,country_month,acled_count_pr_c5,country_month.acled_count_pr_c5,<class 'int'>,False,NaN,NaN,NaN


In [59]:
df.type[0]==float

False

In [35]:
df.apply(lambda row: row.sa_column.type, axis=1).unique()

array([BIGINT(), BIGINT(), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PRECISION(precision=53), DOUBLE_PRECISION(precision=53),
       DOUBLE_PREC

In [44]:
df.sa_column[0].type == sa

NameError: name 'sa' is not defined

In [45]:
import sqlalchemy as sa

In [56]:
sa.types.

AttributeError: module 'sqlalchemy.types' has no attribute 'DOUBLE_PRECISION'

In [14]:
test_cm = pd.DataFrame.c.from_iso(test_cm)

In [48]:
fetch_ids_df('country_month').rename(columns={'id':'c_id'})[['c_id','month_id']]

,c_id,month_id
0,6173,365
1,7625,365
2,9077,365
3,13433,365
4,16337,365
...,...,...
278641,238551,207
278642,244490,218
278643,270921,421
278644,277585,406


In [47]:
ids = fetch_ids_df('country').rename(columns={'id':'c_id'})
ids = ids[ids.c.in_africa].c_id
fetch_ids_df('country_month').rename(columns={'country_id':'c_id'})[['c_id','month_id']]

,c_id,month_id
0,5,365
1,6,365
2,7,365
3,10,365
4,12,365
...,...,...
278641,167,207
278642,183,218
278643,237,421
278644,246,406


In [119]:
test_cm = pd.DataFrame.cm.from_iso(test_cm)

In [15]:
test_cm.cm.fill_spatial_gaps()

,c_id,month_id,iso,value
0,85,401,ITA,1.1
3,85,404,NaN,NaN
4,66,401,SWE,2.1
7,66,404,NaN,NaN
8,64,401,NaN,NaN
11,64,404,NOR,2.2
12,116,401,NaN,NaN
15,116,404,ROU,1.7


In [13]:
test_cm.cm.is_unique

True

In [14]:
countries.c.full_set()

NameError: name 'countries' is not defined

In [ ]:
countries['month_start'] = countries.c.month_start
countries['month_end'] = countries.c.month_end

In [ ]:
set(fetch_ids('month')[0])

In [ ]:
test_cm

In [ ]:
av_c_list = [Country(i) for i in [222,245,246]]

In [ ]:
av_c_list

In [ ]:
Country(1).in_me

In [ ]:
c_sdn = Country.from_iso('SDN')
c_ssd = Country.from_iso('SSD')
c_egy = Country.from_iso('EGY')

In [ ]:
c_sdn,c_ssd,c_egy

In [ ]:
print (c_sdn.month_start,'-',c_sdn.month_end)
print (c_ssd.month_start,'-',c_ssd.month_end)
print (c_egy.month_start,'-',c_egy.month_end)

In [ ]:
av_c_list = [Country(i) for i in [222,245,246]]

In [15]:
test_cm = pd.DataFrame({'iso':     ['SDN','SDN','EGY','EGY','EGY','SSD','SSD'],
                        'month_id':[379,380,378,379,380,379,380],
                        'value':   [1.1,2.1,2.2,1.7,1.1,2.1,4.2]})
test_cm = pd.DataFrame.cm.from_iso(test_cm)
test_cm

,iso,month_id,value,c_id
0,SDN,379,1.1,245
1,SDN,380,2.1,245
2,EGY,378,2.2,222
3,EGY,379,1.7,222
4,EGY,380,1.1,222
5,SSD,379,2.1,246
6,SSD,380,4.2,246


In [17]:
test_cm.cm.is_complete_time_series()

False

In [56]:
test_cm_8 = pd.DataFrame({'iso':     ['RUS','SDN','SSD'],
                        'month_id':[380,380,380],
                        'value':   [1,2,3]})
test_cm_8 = pd.DataFrame.cm.from_iso(test_cm_8)
test_cm_8

,iso,month_id,value,c_id
0,RUS,380,1,65
1,SDN,380,2,245
2,SSD,380,3,246


In [12]:
test_cm_2 = pd.DataFrame({'iso':     ['SDN','SDN','EGY','EGY','SSD','SSD'],
                        'month_id':[379,380,378,380,379,380],
                        'value':   [1.1,2.1,2.2,1.1,2.1,4.2]})
test_cm_2 = pd.DataFrame.cm.from_iso(test_cm_2)
test_cm_2

,iso,month_id,value,c_id
0,SDN,379,1.1,245
1,SDN,380,2.1,245
2,EGY,378,2.2,222
3,EGY,380,1.1,222
4,SSD,379,2.1,246
5,SSD,380,4.2,246


In [52]:
test_cm_3 = pd.DataFrame({'iso':     ['RUS','SDN','EGY','EGY','SSD','SSD'],
                        'month_id':[379,380,401,402,379,380],
                        'value':   [1.1,2.1,2.2,1.1,2.1,4.2]})
test_cm_3 = pd.DataFrame.cm.from_iso(test_cm_3)
test_cm_3

,iso,month_id,value,c_id
0,RUS,379,1.1,65
1,SDN,380,2.1,245
2,EGY,401,2.2,222
3,EGY,402,1.1,222
4,SSD,379,2.1,246
5,SSD,380,4.2,246


In [15]:
test_cm_4 = pd.DataFrame({'iso':     ['SDN','SSD'],
                        'month_id':[401,401],
                        'value':   [1.1,2.1]})
test_cm_4 = pd.DataFrame.cm.from_iso(test_cm_4)
test_cm_4

,iso,month_id,value,c_id
0,SDN,401,1.1,245
1,SSD,401,2.1,246


In [19]:
def is_square(test_square):
    """
    Tests if the 
    """
    min_month = test_square.month_id.min()
    max_month = test_square.month_id.max()

    for cid in test_square.c_id.unique():
        cntry_is_square = True
        test_country = test_square[test_square.c_id == cid]
        if test_country.month_id.min() > min_month and test_country.month_id.min() > Country(cid).month_start:
            return False
        if test_country.month_id.max() < max_month and test_country.month_id.max() < Country(cid).month_end:
            return False
        if len(test_country.index) != max_month-min_month+1:
            low = Country(cid).month_start if Country(cid).month_start > min_month else min_month
            high = Country(cid).month_end if Country(cid).month_end < max_month else max_month
            if high-low+1 != len(test_country.index):
                return False
    return True

In [57]:
is_square(test_cm_8)

True

In [16]:
def is_complete_time_series(test_square, min_month=190, max_month=621):
    if not is_square(test_square):
        return False
    if min_month > test_square.month_id.min() or max_month < test_square.month_id.min():
        return False
    return True

In [59]:
def is_complete_cross_section(test_square, in_africa=False, in_me=False):
    if not is_square(test_square):
        return False
    av_c, _ = fetch_ids('country')
    av_c = [222,246,245]
    min_month = test_square.month_id.min()
    max_month = test_square.month_id.max()
    av_c = [Country(i) for i in av_c]
    av_c = [i for i in av_c if min_month <= i.month_end and max_month >= i.month_start]
    subset_c = []
    if in_africa: subset_c += [i for i in av_c if i.in_africa]
    if in_me: subset_c += [i for i in av_c if i.in_me]
    if len(subset_c) == 0 : subset_c = av_c
    subset_c = set(i.id for i in subset_c) 
    
    if set(test_square.c_id.unique()) == subset_c:
        return True
    return False

In [ ]:
if True or True:
    print (1)

In [31]:
720*360

259200

In [33]:
def full_set(x, land_only=True):
    if land_only:
        ctrl_grids = set(i for i in fetch_ids('priogrid')[0])
    if not land_only:
        ctrl_grids = set(range(1,259201))
    if set(x.pg_id) == ctrl_grids: 
        return True 
    return False

In [4]:
land_set = pd.DataFrame({'pg_id':fetch_ids('priogrid')[0]})

In [5]:
land_set.pg.full_set()

True

In [38]:
full_set(test_pg_1,land_only=True)

False

In [32]:
land_grids_africa = set(i for i in land_grids if i.in_africa)

AttributeError: 'int' object has no attribute 'in_africa'

In [22]:
test_pg_

Priogrid(1)

In [6]:
test_pg_1 = pd.DataFrame({'pg_id': [74804,74805],
                        'value':   [1.1,2.1]})
test_pg_1.pg.lat

0   -38.25
1   -38.25
dtype: float64

In [29]:
def full_set(xxx, full=True):
    test_df = xxx.copy()
    row = [Priogrid(i).row for i in test_df.pg_id]
    col = [Priogrid(i).col for i in test_df.pg_id]
    min_row = min(row); max_row = max(row)
    min_col = min(col); max_col = max(col)
    return min_row, max_row, min_col, max_col

In [30]:
full_set(test_pg_1)

(104, 105, 644, 646)

In [11]:
def is_square(x):
    pass

In [15]:
74805+720

75525

In [24]:
#print(Priogrid(170353).next_up().id)
print(Priogrid(170353).next_left().next_left().next_up().id)
#print(Priogrid(170353).next_left().id)

#print(Priogrid(154542).next_up().id)
#print(Priogrid(154542).next_left().next_up().id)
#print(Priogrid(154542).next_left().id)


171071


In [6]:
test_pg_1 = pd.DataFrame({'pg_id': [
170353,
171072,
171073,
171072,
170352,
155262,
155261,
154541],'value':   [1,1,2,1,2,3,4,2]})
print(test_pg_1.pg.col)
print(test_pg_1.pg.row.min())

0    433
1    432
2    433
3    432
4    432
5    462
6    461
7    461
dtype: int64
215


In [29]:
test_pg_9.pg.row

0    237
1    238
dtype: int64

In [30]:
test_pg_9.pg.col

0    433
1    431
dtype: int64

In [32]:
test_pg_9.pg.get_bbox()

{170351, 170352, 170353, 171071, 171072, 171073}

In [130]:
test_pgm_1 = test_pg_1.copy()
test_pgm_1['val']=0
test_pgm_1 = test_pgm_1.merge(pd.DataFrame({'month_id':[401,402,403],'val':[0,0,0]}), on='val')

In [132]:
test_pgm_1.pgm.is_panel()

True

In [46]:
def is_square(x, only_views_cells=False):
    test_square = x.copy()
    min_row = test_square.pg.row.min()
    max_row = test_square.pg.row.max()
    min_col = test_square.pg.col.min()
    max_col = test_square.pg.col.max()
    square = []
    for row in range(min_row,max_row+1):
        for col in range(min_col,max_col+1):
            square += [Priogrid.from_row_col(row=row,col=col).id]
    square = set(square)
    if only_views_cells:
        views_cells, _ = fetch_ids('priogrid')
        square = set(views_cells).intersection(square)
    return square == set(test_square.pg_id)
            

In [39]:
def is_panel(x):
    test_square = x.copy()
    min_month = test_square.month_id.min()
    max_month = test_square.month_id.max()
    if len(test_square.month_id.unique()) != max_month - min_month + 1 : 
        return False
    first_panel = set(test_square[test_square.month_id==min_month].pg_id.unique())
    for month in test_square.month_id.unique():
        cur_panel = set(test_square[test_square.month_id==month].pg_id.unique())
        if first_panel != cur_panel:
            return False
    return True

In [53]:
def is_complete_cross_section(x,only_views_cells=True):
    if not is_square(x):
        return False
    vc_len = len(fetch_ids('priogrid')[0]) if only_views_cells else 259200
    if len(x.pg_id.unique()) == vc_len:
        return True
    return False

In [49]:
def is_complete_time_series(x,min_month=190,max_month=621):
    if not is_panel(x):
        return False
    if x.test_square.month_id.min() != min_month:
        return False
    if x.test_square.month_id.max() != max_month:
        return False
    return True

In [43]:
land_cells, _ = fetch_ids('priogrid')
land_cells

[49182,
 49183,
 49184,
 49185,
 49186,
 49898,
 49899,
 49900,
 49901,
 49902,
 49903,
 49904,
 49905,
 49906,
 49907,
 49908,
 50616,
 50617,
 50618,
 50619,
 50620,
 50621,
 50622,
 50623,
 50624,
 50625,
 50626,
 50627,
 50628,
 50629,
 50630,
 50631,
 50632,
 50633,
 51078,
 51334,
 51335,
 51336,
 51337,
 51338,
 51339,
 51340,
 51341,
 51342,
 51343,
 51344,
 51345,
 51346,
 51347,
 51348,
 51798,
 52053,
 52054,
 52055,
 52056,
 52057,
 52058,
 52059,
 52060,
 52061,
 52062,
 52063,
 52064,
 52065,
 52066,
 52772,
 52773,
 52774,
 52775,
 52776,
 52777,
 52778,
 52779,
 52780,
 52781,
 52782,
 52783,
 52784,
 53491,
 53492,
 53493,
 53494,
 53495,
 53496,
 53497,
 53498,
 53499,
 53500,
 53501,
 53502,
 53503,
 53504,
 53979,
 54210,
 54211,
 54212,
 54213,
 54214,
 54215,
 54216,
 54217,
 54218,
 54219,
 54220,
 54221,
 54222,
 54223,
 54224,
 54699,
 54930,
 54931,
 54932,
 54933,
 54934,
 54935,
 54936,
 54937,
 54938,
 54939,
 54940,
 54941,
 54942,
 54943,
 55650,
 55651,


In [52]:
len(fetch_ids('priogrid')[0])


64818

In [58]:
test_square = test_cm.copy()
print(is_complete_cross_section(test_cm_8,in_africa=False, in_me=False))

{245, 222, 246}
False


In [ ]:
set([i.id for i in av_c_list if i.month_start <= 370 <= i.month_end])

In [ ]:
av_c, _ = fetch_ids('month')
min(av_c)

In [1]:
from ViewsPandas.scratch import fetch_ids, fetch_ids_df

In [26]:
db_ids = fetch_ids_df('priogrid_month')

In [23]:
db_ids

,id,priogrid_gid,month_id,country_month_id,priogrid_year_id
0,105478131,107742,495,138746.0,946867.0
1,105478132,107742,496,138747.0,946867.0
2,105478133,107742,497,138748.0,946867.0
3,105478134,107742,498,138749.0,946867.0
4,105478135,107742,499,138750.0,946867.0
...,...,...,...,...,...
7223395,189066848,154540,252,NaN,2428491.0
7223396,189066849,170353,252,NaN,3154612.0
7223397,189066850,154542,252,NaN,2428733.0
7223398,189066851,171094,252,NaN,3195631.0


In [21]:
test_pgm_1

,pg_id,value,val,month_id
0,74804,1,0,401
1,74804,1,0,402
2,74804,1,0,403
3,75524,1,0,401
4,75524,1,0,402
5,75524,1,0,403
6,75523,2,0,401
7,75523,2,0,402
8,75523,2,0,403
9,74803,1,0,401


In [29]:
test_c

0    -38.25
1    -38.25
2    -38.25
3    -37.75
4    -37.75
5    -37.75
6    -37.75
7    -37.75
8    -37.75
9    -38.25
10   -38.25
11   -38.25
dtype: float64

In [109]:
db_ids = fetch_ids_df('priogrid_month')[['id','priogrid_gid','month_id']]
test_pgm_2.merge(db_ids,left_on=['pg_id','month_id'],right_on=['priogrid_gid','month_id']).id

0          115764034
1          117856095
2          118830675
3          188484026
4          113255011
             ...    
6501055    130488709
6501056    104892995
6501057    113234475
6501058    123414400
6501059    111377127
Name: id, Length: 6501060, dtype: int64

In [122]:
db_ids = fetch_ids_df('country_month')[['id','country_id','month_id']]
test_cm.merge(db_ids,left_on=['c_id','month_id'],right_on=['country_id','month_id']).id

0    121632
1    101598
2     98838
3    186208
Name: id, dtype: int64

In [121]:
test_cm

,iso,month_id,value,c_id
0,ITA,401,1.1,85
1,SWE,401,2.1,66
2,NOR,402,2.2,64
3,ROU,402,1.7,116


,id,priogrid_gid,month_id
0,105478131,107742,495
1,105478132,107742,496
2,105478133,107742,497
3,105478134,107742,498
4,105478135,107742,499
...,...,...,...
7223395,189066848,154540,252
7223396,189066849,170353,252
7223397,189066850,154542,252
7223398,189066851,171094,252


0          115764034
1          117856095
2          118830675
3          188484026
4          113255011
             ...    
6501055    130488709
6501056    104892995
6501057    113234475
6501058    123414400
6501059    111377127
Name: id, Length: 6501060, dtype: int64

In [126]:
test_pgm_2 = db_ids.sample(frac=.01)
#test_pgm_2
#test_pgm_2[['pg_id']] = test_pgm_2[['priogrid_gid']]
#test_pgm_2 = test_pgm_2[['pg_id','month_id']]

In [128]:
test_pgm_2

,id,country_id,month_id
50386,25812,17,1148
259187,193225,122,443
226733,244072,182,1252
178144,204216,139,200
154389,94809,118,586
...,...,...,...
23086,2634,2,1182
243220,260809,223,1321
68328,57789,33,1355
84606,78884,49,692


In [139]:
test_pgm_1

,pg_id,value,val,month_id
0,170353,1,0,401
1,170353,1,0,402
2,170353,1,0,403
3,154542,1,0,401
4,154542,1,0,402
5,154542,1,0,403
6,171073,2,0,401
7,171073,2,0,402
8,171073,2,0,403
9,171072,1,0,401


In [138]:
pd.DataFrame({'pg':fetch_ids('priogrid')[0]})

,pg
0,49182
1,49183
2,49184
3,49185
4,49186
...,...
64813,249340
64814,249341
64815,249342
64816,249343


In [160]:
month_test = pd.DataFrame({'month_id':[201,202,203,204],'value':[8,8,9,10]})

In [161]:
z = month_test

In [162]:
extent = pd.DataFrame({'month_id':range(z.month_id.min(), z.month_id.max()+1)})
extent.merge(month_test, how='left', on=['month_id'])

,month_id,value
0,201,8
1,202,8
2,203,9
3,204,10


In [5]:
test_pg_9 = pd.DataFrame({'pg_id': [
170353,
171071
],'value':   [1,2]})

In [6]:
bbox = test_pg_9.pg.get_bbox()

In [7]:
pd.DataFrame({'pg_id':list(bbox)}).merge(test_pg_9,how='left',on='pg_id')

,pg_id,value
0,171072,NaN
1,171073,NaN
2,170351,NaN
3,170352,NaN
4,170353,1.0
5,171071,2.0


In [4]:
test_pg_9 = pd.DataFrame({'pg_id': [
170353,
171071
],
'month_id':[409,411],
'value':   [1,2]})

In [5]:
test_pg_9

,pg_id,month_id,value
0,170353,409,1
1,171071,411,2


In [6]:
test_pg_9.pgm.fill_bbox()

,pg_id,month_id,value
0,171072,409,NaN
1,171072,411,NaN
2,171073,409,NaN
3,171073,411,NaN
4,170351,409,NaN
5,170351,411,NaN
6,170352,409,NaN
7,170352,411,NaN
8,170353,409,1.0
9,170353,411,NaN


In [9]:
test_pg_9.pgm.fill_time_gaps()

,pg_id,month_id,value,pgm_id
0,170353,409,1.0,188627474
1,171071,409,NaN,188627989
2,170353,410,NaN,188628651
3,171071,410,NaN,188628147
4,170353,411,NaN,188628997
5,171071,411,2.0,188628904


In [7]:
test_pg_10 = test_pg_9.pgm.db_id()
test_pg_10

,pg_id,month_id,value,pgm_id
0,170353,409,1,188627474
1,171071,411,2,188628904


In [10]:
test_pg_10.pgm.fill_bbox()

,pg_id,month_id,value,pgm_id
0,171072,409,NaN,188627802
1,171072,411,NaN,188628775
2,171073,409,NaN,188628013
3,171073,411,NaN,188628935
4,170351,409,NaN,188627891
5,170351,411,NaN,188628827
6,170352,409,NaN,188627963
7,170352,411,NaN,188629046
8,170353,409,1.0,188627474
9,170353,411,NaN,188628997


In [9]:
pd.DataFrame({'c':101,'d':[8,901]}).append(pd.DataFrame({'c':[100],'d':[99]}))

,c,d
0,101,8
1,101,901
0,100,99
